In [388]:
import pandas as pd
import datetime as dt


In [389]:
df=pd.read_excel('g:\\DA\\finical test lv3\messy.xlsx')

In [390]:
df.head()

,CUst ID,JOIN% DATE,Unnamed: 2,mobiles,FLL NAM
0,1,08/07/19,NaN,84333605993,mIChAel mIcHALek
1,2,09/10/18,NaN,973444062,Andrew Jimenez
2,3,20190630,NaN,338262954,Ann Gow
3,4,2019-09-06 00:00:1567702800,NaN,84966068026,James Chen
4,5,20170812,NaN,84767065885,Dollie Martinez


1,Clean the names of columns to lowercase separatedby “_”, remove any empty column ifnecessary.

In [391]:
df.columns = df.columns.str.strip()  # Xóa khoảng trắng ở đầu và cuối cột
df.columns = df.columns.str.lower().str.replace(' ', '_')
df = df.dropna(axis=1, how='all')

In [392]:
df['cust_id']=df['cust_id'].astype(str)
df['mobiles']=df['mobiles'].astype(str)

2,Change the date column to the same format ‘YYYY-MM-DD’.

In [393]:
def exchange(row):
    if ":" in row['join%_date']:
        row['join%_date'] = row['join%_date'].split(' ')[0]
    else:
        # Chuyển đổi cột 'join%_date' thành đối tượng datetime
        row['join%_date'] = pd.to_datetime(row['join%_date'])
        # Chuyển đổi định dạng của cột thành 'YYYY-MM-DD'
        row['join%_date'] = row['join%_date'].strftime('%Y-%m-%d')
    return row

df = df.apply(exchange, axis=1)
df['join%_date'] = pd.to_datetime(df['join%_date'])


3,Change the name column to the title case (e.g: JasonMraz).

In [394]:
df['fll_nam'] = df['fll_nam'].str.title()


4,Make a new “email” column with form: {last_name}.{first_name}.{id}@yourcompany.com

In [395]:
df['last_name']=df['fll_nam'].str.split().str[-1]
df['first_name']=df['fll_nam'].str.split().str[0]
df['email']=df['last_name']+df['first_name']+'.'+df['cust_id']+'@yourcompany.com'
df=df.drop(['first_name','last_name'],axis=1)

5,Change the phone number column to the format “84......”

In [396]:
df['mobiles'] = df['mobiles'].apply(lambda x: '84' + x if len(x)==9 else ('84'+x[1:] if len(x)==10  else x))

6,Find any duplicated ID and remove those who joinlater.

In [397]:
df.sort_values(by='join%_date',inplace=True)
def removed(data):
    l=[]
    for i,row in data.iterrows():
        if row['cust_id'] not in l:
            l.append(row['cust_id'])
        else :
            data.drop(i,inplace=True)
removed(df)

In [398]:
df.head()

,cust_id,join%_date,mobiles,fll_nam,email
35,36,2017-02-10,84339661824,Gregory Miele,MieleGregory.36@yourcompany.com
37,38,2017-08-02,84344179399,Melvin Pigg,PiggMelvin.38@yourcompany.com
25,26,2017-08-07,84966042581,Charles Garnand,GarnandCharles.26@yourcompany.com
4,5,2017-08-12,84767065885,Dollie Martinez,MartinezDollie.5@yourcompany.com
96,97,2017-08-22,84972812359,Daniel Bentley,BentleyDaniel.97@yourcompany.com


In [399]:
df.loc[df['cust_id']=='9']

,cust_id,join%_date,mobiles,fll_nam,email
33,9,2017-10-21,84285210195,Shelby Dexter,DexterShelby.9@yourcompany.com


7,Filter those who join since 2019 and export to acsv file, delimited by “|”, file name“emp_{report_date}.csv” with report_date = today.

In [400]:
filtered_df = df[df['join%_date'].dt.year == 2019]
# Lấy ngày hôm nay
report_date = dt.date.today().strftime('%Y%m%d')

file_name = f"emp_{report_date}.csv"

filtered_df.to_csv(file_name, sep='|', index=False)
